<div style="width:100%; background-color: #000041"><a target="_blank\" href="http://university.yugabyte.com\"><img src="assets/YBU_Logo.webp" /></a></div><br>

> **YugabyteDB YCQL Development**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/yugabytedb-ycql-development).
>

# Advanced datatypes 
In this notebook, you will learn how to create secondary indexes to not only improve query performance, but also remove unnecessary tables from the data model.

### Import the notebook variables 

> Requirements:
>
> You must first create the variables in the `01_Setup.ipynb` notebook.
>

The following Python cell reads the stored variables created in the `01_Setup.ipynb` notebook. 

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.

In [ ]:
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL
%store -r MY_DATA_DDL_FILE
%store -r MY_DATA_DML_FILE

## Collections
More complex data structures allow YCQL to store data sets that offers more flexibility in its data model capabilities. Very important when a trying to reduce the amount of tables that need to be queried. Run the following cell to add a new column to the category table whose data type is an ordered list with text elements.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Collection: Create a column with a list containing text elements
./bin/ycqlsh --execute "
  ALTER TABLE ks_ybu.tbl_products_by_category 
  ADD warehouse_ids LIST<TEXT>;
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"

#### Collection: SET
Run the following cell to create a new column on the category table. This new column will be a SET with text elements. 

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Collection: Create a column for a set containing text elements
./bin/ycqlsh --execute "
  ALTER TABLE ks_ybu.tbl_products_by_category 
  ADD tags SET<TEXT>;
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"

#### Frozen Collections
Run the following cell to crate a frozen collection. In this example a list was frozen. While this value in this column can be dropped, its elements cannot be manipulated.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Frozen Collection
./bin/ycqlsh --execute "
  ALTER TABLE ks_ybu.tbl_products_by_category 
  ADD store_locations FROZEN<LIST<TEXT>>;
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"

### Time to Live

YCQL offers data expiration. In the context of data modelling, removing "old" or deprecated data can improve database operational costs as well as storage costs.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE TABLE ks_ybu.tbl_wishlist_by_user (
    user_id BIGINT, 
    wishlist_name TEXT, 
    wishlist_id UUID, 
    is_public BOOLEAN, 
    PRIMARY KEY((user_id), wishlist_name)
  ) 
  WITH CLUSTERING ORDER BY (wishlist_name DESC);
"

In the following cell, a row is added to the table with a time-to-live of 5 seconds. Once the time has expired, notice that the row is no longer in the table. This is a TTL scoped to the data row.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  INSERT INTO ks_ybu.tbl_wishlists_by_user(
      user_id,
      name,  
      wishlist_id,
      is_public
    ) VALUES (
      'Mark', 
      'Grocery', 
      2a70494e-6b68-4739-b3e0-ff06aa0a2d67, 
      true
    ) 
    USING TTL 5;
  "  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  INSERT INTO ks_ybu.tbl_wishlists_by_user(
      user_id, 
      name, 
      wishlist_id,
      is_public
    ) VALUES (
      'Mark', 
      'Grocery', 
      2a70494e-6b68-4739-b3e0-ff06aa0a2d67, 
      true
    ) 
    USING TTL 5;
  "  

  ./bin/ycqlsh --execute "
    SELECT * FROM ks_ybu.tbl_wishlist_by_user;
  "  

Wait five seconds before running the following cell. This will verify that the row level data expiration property run successfully.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH
   
./bin/ycqlsh --execute "
  SELECT * FROM ks_ybu.tbl_wishlist_by_user;
" 

If the row is still visible, wait a few more seconds to run the preceding cell. This will verify that the row has expired as expected.

---
# All done!
In this lab, you completed the following:

- Setup
  - Created the `ks_ybu` database with `ycqlsh`
  - Created tables and loaded data using DDL and DML scripts